In [ ]:
from fenics import *
import numpy as np
from numpy import save
from numpy.random import random
import matplotlib.pyplot as plt
from mshr import *
import os
set_log_level(30)

In [ ]:
class PredatorPrey(NonlinearProblem):
    def __init__(self, a, L):
        NonlinearProblem.__init__(self)
        self.L = L
        self.a = a
    def F(self, b, x): assemble(self.L, tensor=b)
    def J(self, A, x): assemble(self.a, tensor=A)

In [ ]:
"""
Define random initial condition
"""
class IC(UserExpression):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    def eval(self, values, x):
        values[0] = 1.0*random() +0.25
        values[1] = 1.0*random() +0.25
    def value_shape(self): return (2,)

In [ ]:
def fenics_field_to_numpy_array(fenics_field, coordinates_size, step, field_name, nb): 
    field_array = np.empty((0, coordinates_size), int)
    lista = []
    for val_x in range(0,coordinates_size):
        for val_y in range(0,coordinates_size):
            try:
                valor=fenics_field(val_x,val_y)
                lista.append(valor)
            except:
                lista.append(0)

            if len(lista) == coordinates_size:
                field_array = np.append(field_array, np.array([lista]), axis=0)
                if val_x < coordinates_size:
                    lista = []
    N_field = "matrix_"  + field_name +"_"+ str(step) + "_" + "nb_" + str(nb) + ".txt"
    np.savetxt(N_field, field_array, delimiter="\t")
    return None

In [ ]:
def grafica(P, N, t):
    plt.figure(figsize=(11, 4))
    
    plt.subplot(1, 2, 1)
    p1 = plot(P)
    p1.set_cmap("seismic")
    plt.title("$P(t={})$".format(t))
    plt.colorbar(p1) 
    
    plt.subplot(1, 2, 2)
    p1 = plot(N)
    p1.set_cmap("gray")
    plt.title("$N(t={})$".format(t))
    plt.colorbar(p1) 
    
    plt.tight_layout(pad=-1.0)
    plt.show()

In [ ]:
def mesh():
    # Define finite diference mesh
    p0 = Point(0.0, 0.0)
    p1 = Point(70,70) 
    mesh = RectangleMesh(p0, p1, 100,100,"right/left")
    return mesh

In [ ]:
def irregular_mesh():
    p0 = Point(0.0, 0.0)
    p1 = Point(70,70) 
    square = Rectangle(p0, p1)
    p2 = Point(10, 5)
    p3 = Point(20,65) 
    rec1 = Rectangle(p2, p3)
    p4 = Point(30,5)
    p5 = Point(40,65) 
    rec2 = Rectangle(p4, p5)
    p6 = Point(50,5)
    p7 = Point(60,65) 
    rec3 = Rectangle(p6, p7)
    domain = square - rec1 -rec2 -rec3
    mesh = generate_mesh(domain,200)
    return mesh

In [ ]:
def function_space(mesh):
    # Work in function space, test and trial functions
    U = FiniteElement("CG", mesh.ufl_cell(), 2)
    W = FunctionSpace(mesh, U * U)
    du   = TrialFunction(W)
    q, p = TestFunctions(W)
    w = Function(W)
    w0 =  Function(W)
    return W,du,q,p,w,w0

In [ ]:
W,du,q,p,w,w0 = function_space(mesh())


In [ ]:
# Split mixed functions
dN, dP = split(du)
N, P = split(w)
N0, P0 = split(w0)

# Apply random inital condition
w_init = IC(element=W.ufl_element(), degree=2)
w.interpolate(w_init)
w0.interpolate(w_init)

In [ ]:
# Time parameters
dt = 5 # step
T = 2000 # end step
t = 0  # zero time
nb = 5

# Set parameters
D_N = 0.015
D_P = 1.0
alpha = 0.64
beta = 0.25
gamma = 0.5

In [ ]:
# writing PDE system in weak form
L0 = N*q -N0*q +D_N*inner(grad(N), grad(q))*dt - alpha*N*(1-N)*(N+P)*q*dt + N*P*q*dt
L1 = P*p -P0*p +D_P*inner(grad(P), grad(p))*dt + beta*P*(N+P)*p*dt - gamma*N*P*p*dt
L  = (L0 + L1) *dx

# Compute directional derivative about u in the direction of du
a = derivative(L, w, du)
problem = PredatorPrey(a, L)
solver = NewtonSolver()
solver.parameters["linear_solver"] = "lu"
solver.parameters["convergence_criterion"] = "incremental"
solver.parameters["relative_tolerance"] = 1e-1

In [ ]:
os.chdir('/home/erick/Desktop/FEniCS/FEniCS-projects/diffution systems/predator-prey/regular_alpha_064')
for block in range(1,nb+1):
    t = 0
    # Apply random inital condition
    w_init = IC(element=W.ufl_element(), degree=2)
    w.interpolate(w_init)
    w0.interpolate(w_init)
    
    N0,P0 = w0.split()
    fenics_field_to_numpy_array(P0, 70, 0, "P", block)
    fenics_field_to_numpy_array(N0, 70, 0, "N", block)
    grafica(P0, N0, 0)
    
    while t < T:
        t += dt
        w0.vector()[:] = w.vector()
        solver.solve(problem, w.vector())
        N,P = w.split()

        fenics_field_to_numpy_array(P, 70, t, "P", block)
        fenics_field_to_numpy_array(N, 70, t, "N", block)

        print("block=", block)        
        print("t=", t)
        #grafica(P, N, t)